In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pymysql
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import datetime
import time
import numpy as np
import pandas as pd
import re

In [3]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument("headless")
driver = webdriver.Chrome(options = options)

In [4]:
# Selenium class에서 성분 추출
def execute_element(selenium_class):
    global results
    for element in selenium_class:
        results = element.text
    return results

In [5]:
# 해당하는 웹툰 리스트를 반복검색하는 함수
def iterator_search(search_platform, webtoon_list, id):
    wbtoon_info = []
    for idx in webtoon_list:
        id += 1
        info = search_platform(idx, driver, id)
        if info==None:
            id -= 1
            pass
        else: 
            wbtoon_info.append(info)
    return wbtoon_info

In [6]:
# 카카오 웹툰 검색
def kakaotoon(webtoon_name, driver, id):
    
    # 크롤링 로그 기록 시작
    f = open("./kakao_crawling_log.txt", 'a', encoding='utf-8')
    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # 카카오 웹툰 페이지를 불러와서 웹툰 제목에 해당하는 결과물 출력
    driver.get('https://webtoon.kakao.com/search')
    time.sleep(2)
    f.write(f"------------------------------------{now}------------------------------------\n")
    search_box = driver.find_element(By.XPATH, '//*[@id="root"]/main/div/div/div/div/div[2]/div/div/div[2]/div/div/input')
    try:
        search_box.send_keys(webtoon_name)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)
        driver.find_element(By.XPATH, '//*[@id="root"]/main/div/div/div/div/div[3]/div/div/div/div/div/a/div[2]').click()
        time.sleep(2)   
    except:
        error = f"'{webtoon_name}'라는 이름의 웹툰은 없습니다!\n"
        f.write(error)
        return print(error)
    def get_info(i):
        author = driver.find_elements(By.XPATH, f'//*[@id="root"]/main/div/div[2]/div/div[1]/div[2]/div[1]/div[1]/div[{i}]/div[2]/div/div')
        authors = execute_element(author)
        
        # author = driver.find_elements(By.XPATH, f'//*[@id="root"]/main/div/div/div/div[1]/div[2]/div[1]/div[1]/div[{i}]/div[2]/div/p[2]')
        # authors = execute_element(author) 

        # genre = driver.find_elements(By.XPATH, f'//*[@id="root"]/main/div/div[2]/div/div[1]/div[2]/div[1]/div[1]/div[{i}]/div[2]/div/div/p[1]')
        # genres = execute_element(genre)
            
        # view = driver.find_elements(By.XPATH,f'//*[@id="root"]/main/div/div[2]/div/div[1]/div[2]/div[1]/div[1]/div[{i}]/div[2]/div/div/p[2]')
        # views = execute_element(view)
    
        # good = driver.find_elements(By.XPATH, f'//*[@id="root"]/main/div/div[2]/div/div[1]/div[2]/div[1]/div[1]/div[{i}]/div[2]/div/div/p[3]')
        # goods = execute_element(good)
            
        driver.find_element(By.XPATH, f'//*[@id="root"]/main/div/div[2]/div/div[1]/div[2]/div[1]/div[1]/div[{i}]/div[2]/div/p[1]').click()
        return authors
    while True:
        try:
            authors = get_info(4)            
            
        except:
            print(f"{webtoon_name}의 element의 XPATH가 일치하지 않습니다!")
            authors = get_info(3)
        break
 
    time.sleep(2)
    story = driver.find_elements(By.XPATH, '//*[@id="root"]/main/div/div[2]/div/div[1]/div[3]/div/div/div[1]/div/div[2]/div/div/div[2]/div/div/div/div/div[2]/div[2]/p')
    storys = execute_element(story).replace('\n',' ')

    keyword = driver.find_elements(By.XPATH, '//*[@id="root"]/main/div/div[2]/div/div[1]/div[3]/div/div/div[1]/div/div[2]/div/div/div[2]/div/div/div/div/div[3]/div[2]')
    keywords = execute_element(keyword).replace('\n','').replace('#',' ')

    time.sleep(2)
    
    df = [id, webtoon_name, authors, storys, keywords]
    f.write(f"{webtoon_name}의 정보 크롤링 성공!\n")
    f.write(f"-------------------------------------------'end'-------------------------------------------\n")
    return df

In [7]:
# 네이버 웹툰 검색
def navertoon(webtoon_name, driver, id):
    # 크롤링 로그 기록 시작
    f = open("./naver_crawling_log.txt", 'a', encoding='utf-8')
    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # 네이버 웹툰 페이지를 불러와서 웹툰 제목에 해당하는 결과물 출력
    f.write(f"------------------------------------{now}------------------------------------\n")
    driver.get(f'https://comic.naver.com/search?keyword={webtoon_name}')
    time.sleep(2)
      
    name = driver.find_elements(By.XPATH, '//*[@id="content"]/div[2]/ul/li[1]/h5/a')
    name_ = str(execute_element(name)).replace(' ','')
    # 검색어에 맞는 웹툰인지 확인
    if webtoon_name == name_:
        driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/ul/li[1]/h5/a').click()
    else: 
        f.write(f'{webtoon_name}의 이름의 네이버웹툰은 검색되지 않습니다.')
        return print("검색결과가 없습니다.")
    time.sleep(2)
    # 각 정보 추출
    author = execute_element(driver.find_elements(By.XPATH, '//*[@id="content"]/div[1]/div[2]/h2/span[3]'))
    good = execute_element(driver.find_elements(By.XPATH, '//*[@id="content"]/div[1]/div[2]/ul/li[5]/div/a/em'))
    story = execute_element(driver.find_elements(By.XPATH, '//*[@id="content"]/div[1]/div[2]/p[1]'))
    genre = execute_element(driver.find_elements(By.XPATH, '//*[@id="content"]/div[1]/div[2]/p[2]/span[1]'))
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="content"]/table/tbody/tr[2]/td[2]/a').click()
    star = execute_element(driver.find_elements(By.XPATH, '//*[@id="topPointTotalNumber"]/strong'))
    vote = execute_element(driver.find_elements(By.XPATH, '//*[@id="topTotalStarPoint"]/span[3]/em'))
    time.sleep(1)
    keyword = execute_element(driver.find_elements(By.XPATH, '//*[@id="comic_view_area"]/div[7]/div[1]')).replace('\n', '')
    time.sleep(1)
    info = [id, webtoon_name,author, good, genre, vote, star, story, keyword]

    f.write(f"{webtoon_name}의 정보 크롤링 성공!\n")
    f.write(f"-------------------------------------------'end'-------------------------------------------\n")
    return info

In [10]:
# mysql db 연결 및 table 생성
def insert_db(platform, wb_list):
    conn = pymysql.connect(
        host='221.138.0.16',
        user='root',
        password='#dp5csv5serv',
        db='gydo_s3_db',
        charset='utf8mb4', 
        autocommit=True, 
        cursorclass=pymysql.cursors.DictCursor 
    )
    
    
    
    cur = conn.cursor()
    if platform == 'naver':
            information = iterator_search(navertoon,wb_list, id=0)
            cur.executemany(f"INSERT INTO {platform}toon(id, title, author, good, genre, vote, star, story, keyword) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)", information)
    elif platform == 'kakao':
            information = iterator_search(kakaotoon,wb_list, id=0)
            cur.executemany(f"INSERT INTO {platform}toon(id, title, author,story, keyword) VALUES (%s,%s,%s,%s,%s)", information)
    else: 
            return print("올바른 플랫폼을 입력하세요.(소문자입력필수)")
    
    conn.commit()
    conn.close()




In [11]:
wb_list = ['ㄱ', '전지적독자시점','화산귀환']
insert_db('naver', wb_list)

신림/남/22
검색결과가 없습니다.
전지적독자시점
화산귀환
